In [3]:
#######################1.1 Notes##########################
#
# +Fixed the bug where the program wouldn't fully close if the new file name was already in use
# +Fixed bug where the program would fail to close if the quit button was hit
# +Removed the need to include .hdf5 when picking a file to save
# +Replaced OpenFile class with File Explorer via askopenfilename command
# +Minor GUI tweaks to allow for future improvements 
# +Attached a proper title to the window
#
##########################################################

import sys
import h5py
from PIL import Image
import numpy as np
from tkinter import *
from tkinter.filedialog import askopenfilename
import re

#Creates a Dictonary to Hold Tags
tagBook = {}
oldFile = ""

class TagWindow:
    def __init__(self, master):
        self.master = master #Creates a basic window
        self.frame = Frame(self.master)
        self.frame.grid()
        
        pageTitle = Label(self.frame, text="Add Item to Library") #Identifies GUI and Purpose
        pageTitle.grid(row = 0, column = 1, columnspan = 2)

        #Creates all blankspace in the window
        blankLbl1 = Label(self.frame, text="")
        blankLbl1.grid(row = 6, column = 0)
        blankLbl2 = Label(self.frame, text="")
        blankLbl2.grid(row = 9, column = 0)
        blankLbl3 = Label(self.frame, text="")
        blankLbl3.grid(row = 13, column = 0)
        blankLbl4 = Label(self.frame, text="")
        blankLbl4.grid(row = 19, column = 0)

        
        #Creates Origin Option

        originLbl = Label(self.frame, text="Origin: ") #Identifies Origin Option
        originLbl.grid(row = 1, column = 0)

        originChoices = ("Field Measurement", "b", "c", "d") #Creates the options for Origin
        self.originV = ""

        self.originOpm = OptionMenu(self.frame, StringVar(self.frame), *originChoices, command = self.originValue) #Creates Origin Dropdown
        self.originOpm.grid(row = 1, column = 3)


        #Creates Category Option

        catLbl = Label(self.frame, text="Category: ") #Identifies Category Option
        catLbl.grid(row = 2, column = 0)

        catChoices = ("a", "b", "c") #Creates the options for Category
        self.catV = ""

        self.catOpm = OptionMenu(self.frame, StringVar(self.frame), *catChoices, command = self.catValue) #Creates Category Dropdown
        self.catOpm.grid(row = 2, column = 3)


        #Creates Class Option

        classLbl = Label(self.frame, text="Class: ") #Identifies Class Option
        classLbl.grid(row = 3, column = 0)
        
        classChoices = ("a", "b", "c") #Creates the options for Class
        self.classV = ""

        self.classOpm = OptionMenu(self.frame, StringVar(self.frame), *classChoices, command = self.classValue) #Creates Class Dropdown
        self.classOpm.grid(row = 3, column = 3)


        #Creates Common Name Option

        commonLbl = Label(self.frame, text="Common Name: ") #Identifies Common Name Option
        commonLbl.grid(row = 4, column = 0)

        commonChoices = ("a", "b", "c") #Creates the options for Common Name
        self.commonV = ""

        self.commonOpm = OptionMenu(self.frame, StringVar(self.frame), *commonChoices, command = self.commonValue) #Creates Common Name Dropdown
        self.commonOpm.grid(row = 4, column = 3)


        #Creates Name Option

        nameLbl = Label(self.frame, text="Name: ") #Identifies Name Option
        nameLbl.grid(row = 5, column = 0)

        self.nameEnt = Entry(self.frame) #Creates Name Entry
        self.nameEnt.grid(row = 5, column = 3)


        #Creates Mark/Mod Option

        mAndMLbl = Label(self.frame, text="Mark/Mod: ") #Identifies Mark/Mod Option
        mAndMLbl.grid(row = 7, column = 0)

        mAndMChoices = ("a", "b", "c") #Creates the options for Mark/Mod Name
        self.mAndMV = ""

        self.mAndMOpm = OptionMenu(self.frame, StringVar(self.frame), *mAndMChoices, command = self.mAndMValue) #Creates Mark/Mod Dropdown
        self.mAndMOpm.grid(row = 7, column = 3)


        #Creates Serial Number Option

        serialLbl = Label(self.frame, text="Serial Number: ") #Identifies Serial Number Option
        serialLbl.grid(row = 8, column = 0)

        self.serialEnt = Entry(self.frame) #Creates Serial Number Entry
        self.serialEnt.grid(row = 8, column = 3)


        #Creates Orientation Option

        orientLbl = Label(self.frame, text="Orientation: ") #Identifies Orientation Option
        orientLbl.grid(row = 10, column = 0)

        orientChoices = ("Up", "Down", "Left", "Right") #Creates the options for Orientation Name
        self.orientV = ""

        self.orientOpm = OptionMenu(self.frame, StringVar(self.frame), *orientChoices, command = self.orientValue) #Creates Orientation Dropdown
        self.orientOpm.grid(row = 10, column = 3)


        #Creates Outside Diameter Option

        diameterLbl = Label(self.frame, text="Outside Diameter (mm): ") #Identifies Outside Diameter Option
        diameterLbl.grid(row = 11, column = 0)

        self.diameterEnt = Entry(self.frame) #Creates Outside Diameter Entry
        self.diameterEnt.grid(row = 11, column = 3)


        #Creates Length Option

        lengthLbl = Label(self.frame, text="Length (mm): ") #Identifies Length Option
        lengthLbl.grid(row = 12, column = 0)

        self.lengthEnt = Entry(self.frame) #Creates Length Entry
        self.lengthEnt.grid(row = 12, column = 3)


        #Creates Condition Option

        conditionLbl = Label(self.frame, text="Condition: ") #Identifies Condition Option
        conditionLbl.grid(row = 14, column = 0)
        
        conditionChoices = ("a", "b", "c") #Creates the options for Condition
        self.conditionV = ""

        self.conditionOpm = OptionMenu(self.frame, StringVar(self.frame), *conditionChoices, command = self.conditionValue) #Creates Condition Dropdown
        self.conditionOpm.grid(row = 14, column = 3)


        #Creates Rotating Band Option

        bandLbl = Label(self.frame, text="Rotating Band: ") #Identifies Rotating Band Option
        bandLbl.grid(row = 15, column = 0)
        
        bandChoices = ("a", "b", "c") #Creates the options for Rotating Band
        self.bandV = ""

        self.bandOpm = OptionMenu(self.frame, StringVar(self.frame), *bandChoices, command = self.bandValue) #Creates Rotating Band Dropdown
        self.bandOpm.grid(row = 15, column = 3)


        #Creates Fins Option

        finsLbl = Label(self.frame, text="Fins: ") #Identifies Fins Option
        finsLbl.grid(row = 16, column = 0)
        
        finsChoices = ("a", "b", "c", "d", "e", "f") #Creates the options for Fins
        self.finsV = ""

        self.finsOpm = OptionMenu(self.frame, StringVar(self.frame), *finsChoices, command = self.finsValue) #Creates Fins Dropdown
        self.finsOpm.grid(row = 16, column = 3)


        #Creates Fuse Option

        fuseLbl = Label(self.frame, text="Fuse: ") #Identifies Fuse Option
        fuseLbl.grid(row = 17, column = 0)
        
        fuseChoices = ("a", "b", "c") #Creates the options for Fuse
        self.fuseV = ""

        self.fuseOpm = OptionMenu(self.frame, StringVar(self.frame), *fuseChoices, command = self.fuseValue) #Creates Fuse Dropdown
        self.fuseOpm.grid(row = 17, column = 3)


        #Creates Spotting Charge Option

        chargeLbl = Label(self.frame, text="Spotting Charge: ") #Identifies Spotting Charge Option
        chargeLbl.grid(row = 18, column = 0)
        
        chargeChoices = ("a", "b", "c") #Creates the options for Spotting Charge
        self.chargeV = ""

        self.chargeOpm = OptionMenu(self.frame, StringVar(self.frame), *chargeChoices, command = self.chargeValue) #Creates Spotting Charge Dropdown
        self.chargeOpm.grid(row = 18, column = 3)


        #Creates Qualifier/Pedigree Option

        qualifierLbl = Label(self.frame, text="Qualifier/Pedigree: ") #Identifies Qualifier/Pedigree Option
        qualifierLbl.grid(row = 20, column = 0)
        
        qualifierChoices = ("a", "b", "c") #Creates the options for Qualifier/Pedigree
        self.qualifierV = ""

        self.qualifierOpm = OptionMenu(self.frame, StringVar(self.frame), *qualifierChoices, command = self.qualifierValue) #Creates Qualifier/Pedigree Dropdown
        self.qualifierOpm.grid(row = 20, column = 3)


        #Creates Buttons

        aplBtn = Button(self.frame, text="Apply", command=self.save) #Creates Apply Button
        aplBtn.grid(row = 21, column = 1)

        canBtn = Button(self.frame, text="Cancel", command=self.exit) #Creates Cancel Button
        canBtn.grid(row = 21, column = 2)
        

    def save(self):
        #Populates the Tag Dictionary
                 
        if self.originV != "":
            tagBook.update({'Origin' : self.originV}) 
        if self.catV != "":
            tagBook.update({'Catagory' : self.catV}) 
        if self.classV != "":
            tagBook.update({'Class' : self.classV}) 
        if self.commonV != "":
            tagBook.update({'Common Name' : self.commonV})  
        if self.nameEnt.get() != "":
            tagBook.update({'Name' : self.nameEnt.get()}) 
        if self.mAndMV != "":
            tagBook.update({'Mark/Mod' : self.mAndMV}) 
        if self.serialEnt.get() != "":
            tagBook.update({'Serial Number' : self.serialEnt.get()})
        if self.orientV != "":
            tagBook.update({'Orientation' : self.orientV}) 
        if self.diameterEnt.get() != "":
            tagBook.update({'Diameter' : self.diameterEnt.get()})
        if self.lengthEnt.get() != "":
            tagBook.update({'Length' : self.lengthEnt.get()})
        if self.conditionV != "":
            tagBook.update({'Condition' : self.conditionV})  
        if self.bandV != "":
            tagBook.update({'Rotating Band' : self.bandV}) 
        if self.finsV != "":
            tagBook.update({'Fins' : self.finsV})  
        if self.fuseV != "":
            tagBook.update({'Fuse' : self.fuseV})
        if self.chargeV != "":
            tagBook.update({'Spotting Charge' : self.chargeV})
        if self.qualifierV != "":
            tagBook.update({'Quailfier/Pedigree' : self.qualifierV})     
        
        #Opens Save Window
        self.newWindow = Toplevel(self.master)
        self.app = SaveFile(self.newWindow)
        
        self.frame.destroy()
    
    def exit(self):
        self.master.destroy()
        
    def originValue(self,value):
        self.originV = value
        
    def catValue(self,value):
        self.catV = value

    def classValue(self,value):
        self.classV = value
    
    def commonValue(self,value):
        self.commonV = value
        
    def mAndMValue(self,value):
        self.mAndMV = value
        
    def orientValue(self,value):
        self.orientV = value
        
    def conditionValue(self,value):
        self.conditionV = value
        
    def bandValue(self,value):
        self.bandV = value
        
    def finsValue(self,value):
        self.finsV = value
        
    def fuseValue(self,value):
        self.fuseV = value
        
    def chargeValue(self,value):
        self.chargeV = value
        
    def qualifierValue(self,value):
        self.qualifierV = value
        
    def getTagBook():
        return self.tagBook

class SaveFile:
    def __init__(self, master):
        self.master = master
        self.frame = Frame(self.master)
        self.frame.grid()
        
        pageTitle = Label(self.frame, text="What would you like to name the new file?") #Identifies GUI and Purpose
        pageTitle.grid(row = 0, column = 0, columnspan = 2)
        
        #Creates File Name Option

        self.fileName = Entry(self.frame) #Creates File Name Entry
        self.fileName.grid(row = 1, column = 0, columnspan = 2)
        
        #Creates Buttons
        
        aplBtn = Button(self.frame, text="Save", command=self.save) #Creates Open Button
        aplBtn.grid(row = 2, column = 0)

        canBtn = Button(self.frame, text="Cancel", command=self.exit) #Creates Cancel Button
        canBtn.grid(row = 2, column = 1)
        self.frame.pack()

    def save(self):

######################Save File Proper######################
        # Save to a new HDF5 file

        #Opens old file
        global oldFile
        f = h5py.File(oldFile, 'r')
        
        # Get the Data
        x = list(f.keys())
    
        # Asks for new file name
        newFile = self.fileName.get()
        
        #Checks for proper file type        
        if re.search(".hdf5", newFile):
            newFile = newFile
        elif re.search(".HDF5", newFile):
            newFile = newFile
        elif re.search(".h5", newFile):
            newFile = newFile
        elif re.search(".H5", newFile):
            newFile = newFile
        else:
            newFile = newFile + ".hdf5"
        
        # Saving Processes
        with h5py.File(newFile, 'w') as hdf:
            
            # Copying existing data
            y = 0
            for y in range(0, len(x)):
                f.copy(x[y], hdf)
            
            #Encoding into ASCII
            asciiList = [n.encode("ascii", "ignore") for n in tagBook.values()]
            hdf.create_dataset('Item Meta Data',(len(tagBook),1), data = asciiList) #tags

        
        f.close()
        
        self.master.destroy()
    
    def exit(self):
        self.master.destroy()

        
def main(): 
   
    root = Tk()
    root.title("HDF5 Tagger")
    root.update_idletasks() #This prevents 'locking' of entry options due to the use of filedialog
    root.withdraw() #To hide stagnate window
    global oldFile
    oldFile = askopenfilename() #File Explorer
    if oldFile != "":
        root.deiconify() #To allow new windows to appear
        app = TagWindow(root)
        
    elif oldFile == "":
        root.destroy()    
        
    root.mainloop()

if __name__ == '__main__':
    main()